In [33]:
import os
import re
import aocd
import string
import pprint
import numpy as np
import pandas as pd
from functools import reduce  # forward compatibility for Python 3
import operator
from aocd import submit
pp = pprint.PrettyPrinter(indent=4)

data = aocd.get_data(day=7, year=2022).split('$')
data


['',
 ' cd /\n',
 ' ls\ndir cmjgvh\ndir czrzl\ndir fcbt\ndir hdh\n259661 hjsbd.mzp\ndir jgrdd\ndir lqblqtng\ndir pgvmpmn\ndir pqqcvcm\ndir zglbptq\n',
 ' cd cmjgvh\n',
 ' ls\ndir hdh\n134565 hdh.sjv\ndir hgrpfmt\n282147 mjtq.ffd\n42343 rvmzv.rtb\ndir sjgvbd\n31468 wgtjmb.thf\n',
 ' cd hdh\n',
 ' ls\n267125 htplc.gdw\n',
 ' cd ..\n',
 ' cd hgrpfmt\n',
 ' ls\n39132 lndwz\n280595 rffmsvdw\n',
 ' cd ..\n',
 ' cd sjgvbd\n',
 ' ls\n26464 ghg.zmq\n1533 zsgdbd.dmm\n',
 ' cd ..\n',
 ' cd ..\n',
 ' cd czrzl\n',
 ' ls\ndir cmh\n242795 hpgnd.mmt\n157748 hps.ptg\n129797 qjrhbjql.zdc\n18290 sfhrzzcd.hwm\n63141 sngg.vdw\n',
 ' cd cmh\n',
 ' ls\ndir nglsj\ndir szs\n76775 vdpqhvrm.mcz\n',
 ' cd nglsj\n',
 ' ls\n307082 cjvph.fvc\n286825 szs.vpj\n',
 ' cd ..\n',
 ' cd szs\n',
 ' ls\n6233 fgpnnvm\n94070 jjc.szq\n',
 ' cd ..\n',
 ' cd ..\n',
 ' cd ..\n',
 ' cd fcbt\n',
 ' ls\n150417 grncq.brq\n',
 ' cd ..\n',
 ' cd hdh\n',
 ' ls\ndir cljdmh\ndir fptsr\ndir schjz\ndir vrrcrhzs\n',
 ' cd cljdmh\n',
 ' ls\ndi

In [32]:
data

['$ cd /',
 '$ ls',
 'dir cmjgvh',
 'dir czrzl',
 'dir fcbt',
 'dir hdh',
 '259661 hjsbd.mzp',
 'dir jgrdd',
 'dir lqblqtng',
 'dir pgvmpmn',
 'dir pqqcvcm',
 'dir zglbptq',
 '$ cd cmjgvh',
 '$ ls',
 'dir hdh',
 '134565 hdh.sjv',
 'dir hgrpfmt',
 '282147 mjtq.ffd',
 '42343 rvmzv.rtb',
 'dir sjgvbd',
 '31468 wgtjmb.thf',
 '$ cd hdh',
 '$ ls',
 '267125 htplc.gdw',
 '$ cd ..',
 '$ cd hgrpfmt',
 '$ ls',
 '39132 lndwz',
 '280595 rffmsvdw',
 '$ cd ..',
 '$ cd sjgvbd',
 '$ ls',
 '26464 ghg.zmq',
 '1533 zsgdbd.dmm',
 '$ cd ..',
 '$ cd ..',
 '$ cd czrzl',
 '$ ls',
 'dir cmh',
 '242795 hpgnd.mmt',
 '157748 hps.ptg',
 '129797 qjrhbjql.zdc',
 '18290 sfhrzzcd.hwm',
 '63141 sngg.vdw',
 '$ cd cmh',
 '$ ls',
 'dir nglsj',
 'dir szs',
 '76775 vdpqhvrm.mcz',
 '$ cd nglsj',
 '$ ls',
 '307082 cjvph.fvc',
 '286825 szs.vpj',
 '$ cd ..',
 '$ cd szs',
 '$ ls',
 '6233 fgpnnvm',
 '94070 jjc.szq',
 '$ cd ..',
 '$ cd ..',
 '$ cd ..',
 '$ cd fcbt',
 '$ ls',
 '150417 grncq.brq',
 '$ cd ..',
 '$ cd hdh',
 '$ ls',
 '

In [38]:


def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value

def add_cur_dir_to_tree(tree, parents):
    pass
    

tree = {}
parents = []
for line in data:
    line = line.strip()#.split('\n')
    #print(line.split('\n'))
    
    if line.startswith('cd'):
        if line == 'cd ..':
            parents = parents[:-1]
        else:
            cur_dir = line.split(' ')[1]
            parents += [cur_dir]
            setInDict(tree, parents, {})
    elif line.startswith('ls'):
        line = line.split('\n')
        sizes = []
        for contents in line[1:]:
            
            if contents.startswith('dir'):
                pass
            else:
                size = int(contents.split(' ')[0])
                sizes += [size]
        setInDict(tree, parents + ['file_sizes'], sizes)
        

pp.pprint(tree)
    

{   '/': {   'cmjgvh': {   'file_sizes': [134565, 282147, 42343, 31468],
                           'hdh': {'file_sizes': [267125]},
                           'hgrpfmt': {'file_sizes': [39132, 280595]},
                           'sjgvbd': {'file_sizes': [26464, 1533]}},
             'czrzl': {   'cmh': {   'file_sizes': [76775],
                                     'nglsj': {'file_sizes': [307082, 286825]},
                                     'szs': {'file_sizes': [6233, 94070]}},
                          'file_sizes': [242795, 157748, 129797, 18290, 63141]},
             'fcbt': {'file_sizes': [150417]},
             'file_sizes': [259661],
             'hdh': {   'cljdmh': {   'blsrsqz': {'file_sizes': [275130]},
                                      'file_sizes': [   21876,
                                                        287047,
                                                        33008,
                                                        135650,
                 

In [39]:

def get_folder_sizes(tree):
    size = 0
    for k in tree.keys():
        if k == 'file_sizes':
            size += sum(tree[k])
        elif k == 'folder_size':
            pass
        else:
            size += get_folder_sizes(tree[k])
    tree['folder_size'] = size
    return size
get_folder_sizes(tree)

45174025

In [40]:
def get_folder_sizes_under_100000(tree):
    sizes = 0
    for k in tree.keys():
        if k == 'file_sizes':
            pass
        elif k == 'folder_size':
            if tree[k] < 100000:
                sizes += tree[k]
        else:
            sizes += get_folder_sizes_under_100000(tree[k])

    return sizes
get_folder_sizes_under_100000(tree)

1077191

In [46]:
30000000 - (70000000 - tree['/']['folder_size'])

5174025

In [50]:
global closest_size 
closest_size = 30000000

def get_folder_sizes_just_over_5174025(tree):
    global closest_size 
    sizes = 0
    for k in tree.keys():
        if k == 'file_sizes':
            pass
        elif k == 'folder_size':
            if tree[k] - 5174025 > 0:
                if tree[k] < closest_size:
                    closest_size = tree[k]
        else:
            get_folder_sizes_just_over_5174025(tree[k])

    return closest_size
get_folder_sizes_just_over_5174025(tree), closest_size

(5649896, 5649896)